In [68]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations

%load_ext autoreload
%autoreload 2
%matplotlib inline

warnings.filterwarnings('ignore')

# Load data
ust_futures = pd.read_csv('raw_futures_prices.csv', index_col=0, parse_dates=[0]).dropna(how='any', axis=0)

# Potential pairs.
pairs = list(combinations(ust_futures.columns, 2))

# Define spreads
# TODO LATER: Pull spreads from Chi-Wai's email.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
from collections import defaultdict
from more_itertools import windowed

import v2.copulas.fitting as fitting
import v2.backtest.hedging as hedging
import v2.backtest.signal_ as signal_

# PnL analysis for each pair.
yearly_copulas = {}

# Make a list of years.
years = ['2018', '2019', '2020', '2021', '2022']
years_windowed = list(windowed(years, 3))

for train_start, train_end, test_end in years_windowed:
    # Fit the copulas -- index by trading year for the
    # dictionary.
    yearly_signal_obj = signal_.SignalGeneration(ust_futures, pairs, train_start, train_end, test_end, test_end,copula_type='mixed')
    yearly_copulas[test_end] = yearly_signal_obj

In [80]:
import backtest.backtester as backtest

prep_backtest = {}

for year, signal_obj in yearly_copulas.items():
    prices = signal_obj.prices
    sigs = signal_obj.calc_signals()
    prices = prices.loc[year]
    for col in sigs.columns:
        prices[col] = signal_obj.generate_signals(sigs[col]).values

    #prices[sigs.columns] = sigs.values
    prep_backtest[year]= prices.copy()

In [89]:
# Run backtest
YEAR = '2022'

agg_results = pd.DataFrame()

for pair in pairs:
    train_data = yearly_copulas[YEAR].prices[[pair[0], pair[1]]]
    pair_data = prep_backtest[YEAR][[pair[0], pair[1], pair]]
    pair_data = pair_data.rename({pair: 'Signal'}, axis=1)
    pair_data['Betas'] = 1
    res = backtest.backtest(pair_data)
    agg_results[pair] = res['Total PnL']
agg_results = (agg_results + 10_000).pct_change().dropna()
print(agg_results.mean()*252**0.5/agg_results.std())
#agg_results.plot()

(TU, FV)     1.570110
(TU, TY)     1.664886
(TU, UXY)    1.681322
(TU, US)     1.815963
(TU, WN)     2.397748
(FV, TY)     1.101572
(FV, UXY)    1.439778
(FV, US)     1.293597
(FV, WN)     1.588300
(TY, UXY)    1.243457
(TY, US)     1.042754
(TY, WN)     1.213678
(UXY, US)    1.637854
(UXY, WN)    0.882616
(US, WN)     0.608913
dtype: float64
